In [1]:
import os

In [2]:
%pwd

'c:\\DataScience\\Projects\\folder\\AIDRP-AI_Driven_Diabetes_Readmission_Prevention\\research'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path : Path
    test_data_path : Path
    model_name: str
    iterations: int
    learning_rate: float
    depth: int
    l2_leaf_reg: float
    bootstrap_type: str
    random_strength: float
    bagging_temperature: float
    od_type: str
    od_wait:int
    target_column: str
  


In [6]:
from src.AIDRP.constants import *
from src.AIDRP.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.CatBoostClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
    
        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            iterations=params.iterations,
            learning_rate=params.learning_rate,
            depth=params.depth,
            l2_leaf_reg=params.l2_leaf_reg,
            bootstrap_type=params.bootstrap_type,
            random_strength=params.random_strength,
            bagging_temperature=params.bagging_temperature,
            od_type=params.od_type,
            od_wait=params.od_wait,
            target_column = schema.name
        )

        return model_trainer_config
  

In [9]:
import pandas as pd
import os
from src.AIDRP.logging import logger
import joblib
import catboost

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        params={'iterations': self.config.iterations, 'learning_rate': self.config.learning_rate,
                 'depth': self.config.depth, 'l2_leaf_reg': self.config.l2_leaf_reg, 
                 'bootstrap_type': self.config.bootstrap_type, 'random_strength': self.config.random_strength,
                   'bagging_temperature': self.config.bagging_temperature, 'od_type': self.config.od_type, 'od_wait': self.config.od_wait}

        cb = catboost.CatBoostClassifier(**params,random_state=42,verbose=False)
        cb.fit(train_x, train_y)

        joblib.dump(cb, os.path.join(self.config.root_dir, self.config.model_name))

        

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-02-12 20:58:50,987 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-02-12 20:58:50,995 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-02-12 20:58:51,009 : INFO : common : yaml file: schema.yaml loaded successfully]
[2024-02-12 20:58:51,019 : INFO : common : created directory at: artifacts]
[2024-02-12 20:58:51,020 : INFO : common : created directory at: artifacts/model_trainer]
